In [8]:
import functools

In [9]:
# name = "trifold"

# nodes = [
#     (1, [2, 6]),
#     (2, [3]),
#     (3, [7, 11, 4]),
#     (4, [5, 16, 12]),
#     (5, [6]),
#     (6, []),
#     (7, [8]),
#     (8, [9]),
#     (9, [10]),
#     (10, [11, 13]),
#     (11, []),
#     (12, [13]),
#     (13, [14]),
#     (14, [15]),
#     (15, [16]),
#     (16, []),

# ]

# robbers = [
#     "n1",
#     ]

# cops = [
#     "n8",
#     "n15",
# ]

In [10]:
name = "line_chase"

nodes = [
    (1, [2]),
    (2, [3]),
    (3, [4]),
    (4, [5]),
    (5, [6]),
    (6, []),
]

robbers = [
    "n6",
    ]

cops = [
    "n1",
]

In [11]:
# variants
friendly = True

In [12]:
counts = [0] * len(nodes)
rob_objects = functools.reduce(lambda a, b: a + b, ["rob" + str(i + 1) + " " for i in range(len(robbers))])
cop_objects = functools.reduce(lambda a, b: a + b, ["cop" + str(i + 1) + " " for i in range(len(cops))])
node_objects = functools.reduce(lambda a, b: a + b, ["n" + str(i + 1) + " " for i in range(len(nodes))])
edges = []

In [13]:
with open(f"problems/problem_{name}.pddl", "w") as file1:
    file1.write(f"(define (problem cops_robbers_{name})\n")
    file1.write('\t(:domain cops_robbers)\n')
    file1.write("\t\n")
    file1.write("\t(:objects\n")
    file1.write(f"\t\t{rob_objects}- robber\n")
    file1.write(f"\t\t{cop_objects}- cop\n")
    file1.write(f"\t\t{node_objects}- location\n")
    file1.write("\t)\n")
    file1.write("\t\n")
    file1.write("\t(:init\n")
    for i in range(len(nodes)):
        for j in range(len(nodes[i][1])):
            edges.append((nodes[i][0], nodes[i][1][j]))
            file1.write(f"\t\t(edge n{nodes[i][0]} n{nodes[i][1][j]})\n")
            file1.write(f"\t\t(edge n{nodes[i][1][j]} n{nodes[i][0]})\n")
            file1.write(f"\t\t\n")
            counts[nodes[i][0] - 1] += 1
            counts[nodes[i][1][j] - 1] += 1
        file1.write(f"\t\t(num_connections n{i + 1} num{counts[i]})\n")
        file1.write(f"\t\t\n")
    for i in range(len(robbers)):
        file1.write(f"\t\t(at rob{i + 1} {robbers[i]})\n")
    file1.write(f"\t\t\n")
    for i in range(len(cops)):
        file1.write(f"\t\t(at cop{i + 1} {cops[i]})\n")
    file1.write(f"\t\t\n")
    if friendly:
        file1.write(f"\t\t(friendly)\n")
    file1.write("""
    )

    (:goal (and
        (done)
    ))
        
)
    """)
print("problem created")

problem created


In [14]:
with open(f"domains/domain_{name}.pddl", "w") as file1:
    file1.write(f"""(define (domain cops_robbers)

(:requirements
    :adl
    :negative-preconditions
    :non-deterministic)

    (:types
        location entity number - object
        node - location
        cop robber - entity
    )
    
    (:predicates
        ;Designates an entity occupies a node
        (at ?p - entity ?x - location)
        ;Designates which nodes are connected
        (edge ?x ?y - location)
        ;Designates which cop has to move
        (turn ?x - cop)
        
        ;caught is true if the robber occupies the same square as a cop
        (caught)
        ;designates if the robber wants to be caught
        (friendly)
        ;designates if the robber wants to be caught
        (friendly)
        ;survived is true if the robber survives. It acts as a flag for the game to end
        (survived)
        ;done is a flag to end the game if the robber is not caught and has survived
        (done)
        ;nil does nothing. used to make oneofs do nothing
        (nil) 
    )\n\n""")

    file1.write("""\t;The robber makes a move
    (:action robber_move
        ;given 2 locations
        :parameters (?x ?y - location)

        :precondition (and
            ;if the robber is at the first location
            (at rob1 ?x)
            ;the second location is reachable
            (edge ?x ?y)
            ;and there are no cops which can make moves
            (forall (?c - cop) (not (turn ?c)))
        )
                
        :effect (and
            ;check if the new location moves the robber onto a cop
            (when (exists (?c - cop) (at ?c ?y)) (caught))
            ;remove the robber from the first node
            (not (at rob1 ?x))
            ;place him at the second node
            (at rob1 ?y)
            ;make it the cops turn
            (forall (?c - cop)  
                (turn ?c)
            )

            ;non deterministically end the game or do nothing
            (oneof
                (survived)
                (nil)
            )
        )
    )
                
    ;The robber stays in place
    (:action robber_stay
        ;given a location
        :parameters (?x - location)
        :precondition (and
            ;if the robber is at that location
            (at rob1 ?x)
            ;and no cops want to make a move
            (forall (?c - cop) (not (turn ?c)))
        )
        :effect (and
            ;check if a cop is at the same locaiton
            (when (exists (?c - cop) (at ?c ?y)) (caught))

            ;make it the cops turn
            (forall (?c - cop)  
                (turn ?c)
            )

            ;non deterministically end the game or do nothing
            (oneof
                (survived)
                (nil)
            )
        )
    )\n\n""")

    for node in [x[0] for x in nodes]:
        file1.write(f"""\t;A cop move at node {node}
    (:action cop_move{node}
        ;given a cop and a node
        :parameters (?c - cop)

        :precondition (and
            ;if it is the cops turn
            (turn ?c)
            ;the cop is at the node
            (at ?c n{node})
        )
        
        ;move the cop to a non deterministic an adjacent node
        :effect (and
            ;the cop no longer has a turn
            (not (turn ?c))
            ;the cop is not at the first node
            (not (at ?c n{node}))\n""")

        if counts[node - 1] <= 1:
            file1.write("""
            ;the cop moves to the edge""")
            for left, right in edges:
                if node == left:
                    file1.write(f"""
            (and 
                (at ?c n{right})
                (when (at rob1 n{right}) (caught))
            )\n""")
                if node == right:
                    file1.write(f"""
            (and 
                (at ?c n{left})
                (when (at rob1 n{left}) (caught))
            )\n""")
            file1.write("""
        )   
    )\n\n""")

        else:

            file1.write("""
            ;the cop non deterministically moves between one of the edges
            (oneof\n""")

            for left, right in edges:
                if node == left:
                    file1.write(f"""
                (and 
                    (at ?c n{right})
                    (when (at rob1 n{right}) (caught))
                )\n""")
                elif node == right:
                    file1.write(f"""
                (and 
                    (at ?c n{left})
                    (when (at rob1 n{left}) (caught))
                )\n""")
                
            file1.write("""            )
        )   
    )\n\n""")

    file1.write("""
    ;when the cops catch the robbers, end the game
    (:action game_terminate
        :parameters ()
        ;check if survived and not caught
        ;not caught incentivies the robber to survive as long as possible
        ;surivived ensures the game can run indefinitly
        ;combined the robber will try to survive indefinitly
        ;We don't want a definitive way to end the game because we want a plan for the whole system,
        ;not just a way to end the game
        :precondition (and
            ;You don't want friendly robbers to stall for a chance at winning
            (not (friendly))
            (survived)
            (not (caught))
        )
        ;when satisfied end the game
        :effect (and
            (done)            
        )
    )        
                
    ;Friendly variation of the game
    ;The game ends if the robber is caught
    ;The robber has no incentive to run
    ;The planner wants to end the game
    (:action game_terminate
        :parameters ()
        :precondition (and
            ;If friendly
            (friendly)
            ;And caught
            (caught)
        )
        ;end the game
        :effect (and
            (done)   
        )
    )
)""")

print("domain created")

domain created
